Imports

In [1]:
import torch
import os
import sys
from torchvision import transforms
from IPython.display import Image 
import progressbar


script_root_dir = '../'
if script_root_dir not in sys.path:
    sys.path.append(script_root_dir)
    
from train import dataLoader
from train.configParser import ConfigParser
from myhelpers.color_PCA import Color_PCA
from myhelpers import TrialStatistics

Parameters

In [2]:
experimentsPath="/home/elhamod/HGNN/experiments/"
experimentName="DifferentDatasetSizes"
experimentHash="5f5e2a09e703a0e0d6d0cbfbacb6123312be7daf07ba9cf91fcea3e6"
dataPath="/data/BGNN_data/"

imageDimension = 224

applyHorizontalFlip = True
applyHorizontalFlipProbability = 0.1

applyTranslation=True
applyTranslationAmount = 0.1

applyColorPCA=True
colorPCAperturbationMagnitude = 0.1

numOfAugmentedVersions = 10

cuda = 7

In [3]:
# set cuda
if torch.cuda.is_available():
    torch.cuda.set_device(cuda)

Load dataset

In [4]:
# Get experiment params.
config_parser = ConfigParser(experimentsPath, dataPath, experimentName)
experiment_params = None
for params_ in config_parser.getExperiments():
    if experimentHash == TrialStatistics.getTrialName(params_):
        experiment_params = params_
        break
if experiment_params is None:
    print("Could not find experiment!", experimentHash)
    raise

# Get the dataset
experimentPathAndName = os.path.join(experimentsPath, experimentName)
datasetManager = dataLoader.datasetManager(experimentPathAndName)
datasetManager.updateParams(config_parser.fixPaths(experiment_params))
dataset = datasetManager.getDataset()

# disable normalization before augmentation.
dataset.toggle_image_loading(augmentation=False, normalization=False)

# Create appropriate loaders
loaderPerImage = torch.utils.data.DataLoader(dataset, batch_size=1)
datasetSize = len(dataset)
loaderAllImages = torch.utils.data.DataLoader(dataset, batch_size=datasetSize)
print("number of images", datasetSize)

Creating dataset...


Loading images: 100%|██████████| 3800/3800 [00:57<00:00, 66.32it/s, fileName=/data/BGNN_data/INHS_cropped/images/INHS_FISH_63588_224.jpg]  


Creating dataset... Done.
number of images 3800


Generate data

In [5]:
color_pca = Color_PCA(loaderAllImages, colorPCAperturbationMagnitude)

with progressbar.ProgressBar(max_value=datasetSize) as bar:
    k=1
    for batch in loaderPerImage:
        original = batch['image'].squeeze()
        fileName = batch['fileName'][0]
        prefix, ext = os.path.splitext(fileName)

        if torch.cuda.is_available():
            original = original.cpu()
        
        # Generate a scaled version
        savedFileName = os.path.join(dataset.csv_processor.get_image_full_path(), prefix+"_"+str(imageDimension)+ext)
        scaled = transforms.ToPILImage()(original)
        scaled.save(savedFileName)
        print(savedFileName, "saved!")

        # Generate augmented versions
        for j in range(numOfAugmentedVersions):
            image = transforms.ToPILImage()(original)

            if applyHorizontalFlip:
                image = transforms.RandomHorizontalFlip(p=applyHorizontalFlipProbability)(image)
    #             display(image)

            if applyTranslation:
                RGBmean = (int(0.485*255), int(0.456*255), int(0.406*255))
                image = transforms.RandomAffine(degrees=0, translate=(applyTranslationAmount,
                                                                      applyTranslationAmount),
                                               fillcolor=RGBmean)(image)
    #             display(image)

            if applyColorPCA:
                image = transforms.ToTensor()(image).unsqueeze(0)
                image = transforms.Lambda(color_pca.perturb_color)(image).type(torch.float32) 

                image = transforms.ToPILImage()(image.squeeze())
    #             display(image)


            savedFileName = os.path.join(dataset.csv_processor.get_image_full_path(), prefix+"_"+str(imageDimension)+"_aug"+str(j)+ext)
            image.save(savedFileName)
            print(savedFileName, "saved!")
            
        bar.update(k)
        k = k + 1

    
    

Transforming images: 100%|██████████| 3800/3800 [00:16<00:00, 227.47it/s]
  0% (3 of 3800) |                       | Elapsed Time: 0:00:00 ETA:   0:02:21

/data/BGNN_data/INHS_cropped/images/INHS_FISH_99690_224.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_99690_224_aug0.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_99690_224_aug1.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_99690_224_aug2.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_99690_224_aug3.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_99690_224_aug4.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_99690_224_aug5.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_99690_224_aug6.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_99690_224_aug7.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_99690_224_aug8.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_99690_224_aug9.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_47613_224.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_47613_224_aug0.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_47613_224_aug1.jpg saved!
/d

  0% (7 of 3800) |                       | Elapsed Time: 0:00:00 ETA:   0:03:06

/data/BGNN_data/INHS_cropped/images/INHS_FISH_46357_224_aug5.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_46357_224_aug6.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_46357_224_aug7.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_46357_224_aug8.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_46357_224_aug9.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_74231_224.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_74231_224_aug0.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_74231_224_aug1.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_74231_224_aug2.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_74231_224_aug3.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_74231_224_aug4.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_74231_224_aug5.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_74231_224_aug6.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_74231_224_aug7.jpg save

  0% (9 of 3800) |                       | Elapsed Time: 0:00:00 ETA:   0:03:12

...

/data/BGNN_data/INHS_cropped/images/INHS_FISH_108140_224_aug5.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_108140_224_aug6.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_108140_224_aug7.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_108140_224_aug8.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_108140_224_aug9.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_68227_224.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_68227_224_aug0.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_68227_224_aug1.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_68227_224_aug2.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_68227_224_aug3.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_68227_224_aug4.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_68227_224_aug5.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_68227_224_aug6.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_68227_224_aug7.jpg

 99% (3796 of 3800) |################### | Elapsed Time: 0:03:41 ETA:   0:00:00

/data/BGNN_data/INHS_cropped/images/INHS_FISH_102996_224_aug0.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_102996_224_aug1.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_102996_224_aug2.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_102996_224_aug3.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_102996_224_aug4.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_102996_224_aug5.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_102996_224_aug6.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_102996_224_aug7.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_102996_224_aug8.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_102996_224_aug9.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_87361_224.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_87361_224_aug0.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_87361_224_aug1.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_87361_224_aug

100% (3800 of 3800) |####################| Elapsed Time: 0:03:41 Time:  0:03:41


/data/BGNN_data/INHS_cropped/images/INHS_FISH_92277_224_aug7.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_92277_224_aug8.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_92277_224_aug9.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_77972_224.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_77972_224_aug0.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_77972_224_aug1.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_77972_224_aug2.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_77972_224_aug3.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_77972_224_aug4.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_77972_224_aug5.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_77972_224_aug6.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_77972_224_aug7.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_77972_224_aug8.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_77972_224_aug9.jpg save